In [1]:
from paho.mqtt import client as mqtt_client
from datetime import datetime
import pandas as pd
import os.path
import random

In [2]:
broker = 'broker.hivemq.com'
port = 1883
client_id = f'n1-sd-hidroponia-{random.randint(0, 1000)}'
topics = {
    'n1_sd_hidro_ph_temp_data': 'Temperatura e PH da agua (Dados)',
}
data = {
    'Type': [],
    'Temperature': [],
    'PH': [],
    'Date': [],
}

In [3]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Conectado ao MQTT Broker!")
    else:
        print("Falha ao se conectar, código %d\n", rc)

In [4]:
client = mqtt_client.Client(client_id)
client.on_connect = on_connect
client.connect(broker, port)
client.loop_start()

Conectado ao MQTT Broker!


In [5]:
def subscribe(topic):
    def on_message(client, userdata, msg):
        print(f"Recebido `{msg.payload.decode()}` para `{topics[msg.topic]}`")
        now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        msgSplit = msg.payload.decode().split(',')
        file_exists = os.path.exists('data.csv')
        
        if file_exists:
            appendDt = [[ msg.topic, msgSplit[0], msgSplit[1], now ]]
        else:
            data['Type'].append(msg.topic)
            data['Temperature'].append(msgSplit[0])
            data['PH'].append(msgSplit[1])
            data['Date'].append(now)
            appendDt = data
        
        df = pd.DataFrame(appendDt)
        df.to_csv('data.csv', mode='a', index=False, header=not file_exists)
        

    client.subscribe(topic)
    client.on_message = on_message

In [6]:
subscribe('n1_sd_hidro_ph_temp_data')

Recebido `10,7` para `Temperatura e PH da agua (Dados)`
Recebido `5,4` para `Temperatura e PH da agua (Dados)`
Recebido `38,14` para `Temperatura e PH da agua (Dados)`
Recebido `22,10` para `Temperatura e PH da agua (Dados)`
Recebido `12,9` para `Temperatura e PH da agua (Dados)`
Recebido `18,11` para `Temperatura e PH da agua (Dados)`
Recebido `28,13` para `Temperatura e PH da agua (Dados)`
Recebido `39,15` para `Temperatura e PH da agua (Dados)`
Recebido `10,6` para `Temperatura e PH da agua (Dados)`
Recebido `9,5` para `Temperatura e PH da agua (Dados)`
Recebido `14,12` para `Temperatura e PH da agua (Dados)`
Recebido `19,14` para `Temperatura e PH da agua (Dados)`
Recebido `21,16` para `Temperatura e PH da agua (Dados)`
Recebido `11,8` para `Temperatura e PH da agua (Dados)`


In [ ]:
client.loop_stop()